In [ ]:
%matplotlib

# Import required packages
import threading
from threading import Thread
import time
import re
import logging
import os
import serial as pyserial
import sys
import glob

# from tkinter import ttk
# import tkinter as tk
# from tkinter import scrolledtext
# from tkinter import messagebox as msg
# import tkinter.font as tkfont

from IPython.display import display, Markdown, clear_output, Image, HTML
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, Layout, Box, Button, Label, FloatText, Textarea, Dropdown

import matplotlib
import matplotlib.figure as figure
import matplotlib.animation as animation
from matplotlib import rc
import matplotlib.dates as mdates
from matplotlib.ticker import FormatStrFormatter
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
#matplotlib.use("nbagg")
import matplotlib.pyplot as plt
import numpy as np

# Custom Classes
#from meerkat import Meerkat, Response

In [ ]:
rc('animation', html='html5')

In [ ]:
# Maximum number of data points to store for each channel's data arrays.
max_elements = 300
ch0_timestamps = []
ch4_timestamps = []
ch6_timestamps = []

ch0_voltages = []
ch4_voltages = []
ch6_voltages = []

ch0_values = []
ch4_values = []
ch6_values = []

safari_running = False
test_mode_enabled = False
motor_running = False

In [ ]:
# Class for Serial connectivity
class Serial:
    def __init__(self):
        self.BAUD = 115200  # Serial Baud Rate
        self.serial = None
        self.serial_buffer = ""    
        
        self.serial_connected = False
        
        self.serial_ports = self.__serial_port_scan()
        self.selected_serial_port = self.serial_ports[0]
        
        
        self.serial_port_list = widgets.Dropdown(
                                options=self.serial_ports,
                                value=self.selected_serial_port,
                                description='Serial Port:')
        self.serial_port_list.observe(self.on_serial_port_change)
        
        self.serialButton = widgets.Button(description = 'Connect')   
        self.serialButton.on_click(self.on_serial_button_clicked)

        self.thread = threading.Thread(target=self.read_serial)
        self.thread.daemon = True
            
    def on_serial_button_clicked(self, arg):
        if self.serial_connected:
            self.disconnect_serial()
        else:
            self.connect_serial(self.selected_serial_port, self.BAUD)

    # Function for opening a serial connection on the specified port
    def connect_serial(self, port, baud):
        """ The function initiates the Connection to the UART device with the specified Port.
        The radio button selects the platform, as the serial object has different key phrases
        for Linux and Windows. Some Exceptions have been made to prevent the app from crashing,
        such as blank entry fields and value errors, this is due to the state-less-ness of the
        UART device, the device sends data at regular intervals irrespective of the master's state.
        The other Parts are self explanatory.

        :param port: Serial port to connect to.
        :returns:
            0 if the serial connection is successfully opened.
            -1 if the connection fails to open.
        """

        print("Attempting to open port {}".format(port))

        try:
            self.serial = pyserial.Serial(port, baud, timeout=0, writeTimeout=0)  # ensure non-blocking
            self.serial_connected = True
            self.serial_port_list.disabled = True
            print("Serial Port Opened")
            self.serialButton.description = "Disconnect"
            # Start a thread to handle receiving serial data
            self.thread.start()

            # Kill any running processes
            self.__serial_ctrl_c()
            #time.sleep(0.25)
            self.__login()
            return 0
        except Exception as e:
            logging.exception(e)
            print("Cant Open Specified Port")
            return -1
        
    def disconnect_serial(self):
        """ This function is for disconnecting and quitting the application.
            Sometimes the application throws a couple of errors while it is being shut down, the fix isn't out yet

        :returns:
            0 if the serial connection is successfully closed.
            -1 if the connection fails to close.
        """

        try:
            self.serial.close()
            self.serial_connected = False
            self.serial_port_list.disabled = False
            self.serialButton.description = "Connect"
            print("Serial Port Closed")
            return 0

        except AttributeError:
            print("Closed without Using it -_-")
            return -1

    def read_serial(self):
        # Infinite loop is okay since this is running in it's own thread.
        while True:
            if self.serial_connected:
                try:
                    c = self.serial.read().decode('unicode_escape')  # attempt to read a character from Serial

                    # was anything read?
                    if len(c) == 0:
                        pass

                    # check if character is a delimeter
                    if c == '\r':
                        c = ''  # don't want returns. chuck it

                    if c == '\n':
                        self.serial_buffer += "\n"  # add the newline to the buffer

                        # Parse the received serial data since we've received a full line
                        #self.parse_serial(meerkat.serial_buffer)
                        if "login" in self.serial_buffer:
                            #self.label_serial_status.configure(text="Connected", fg="green")
                            self.__login()
                        elif "AD7124 error" in self.serial_buffer:
                            self.__serial_ctrl_c()
                        elif "channel" in self.serial_buffer \
                                and "timestamp" in self.serial_buffer \
                                and "voltage" in self.serial_buffer \
                                and "code" in self.serial_buffer:
                                    self.update_plot_data(self.serial_buffer)
                        elif "FAULT DETECTED!" in self.serial_buffer:
                            print("Serial Fault Detected!")

                        self.serial_buffer = ""  # empty the buffer
                    else:
                        self.serial_buffer += c  # add to the buffer

                except Exception as e:
                        logging.exception(e)
                        pass            

    # Function for parsing the received serial string
    def parse_serial(self, serial_string):
        if "login" in serial_string:
            #self.label_serial_status.configure(text="Connected", fg="green")
            self.__login()
        elif "AD7124 error" in serial_string:
            self.__serial_ctrl_c()
        elif "channel" in serial_string \
                and "timestamp" in serial_string \
                and "voltage" in serial_string \
                and "code" in serial_string:
                    meerkat.update_plot_data(serial_string)
        elif "FAULT DETECTED!" in serial_string:
            print("Serial Fault Detected!")
            
    def update_plot_data(self, data_string):
        """ Parse the data_string into key values and store the data into arrays

        :param data_string: String to parse into relevant data
        :return: Nothing
        """
        
        global ch0_timestamps
        global ch4_timestamps
        global ch6_timestamps

        global ch0_voltages
        global ch4_voltages
        global ch6_voltages

        global ch0_values
        global ch4_values
        global ch6_values

        data_string = data_string.replace("channel,", '')
        data_string = data_string.replace("timestamp,", '')
        data_string = data_string.replace("voltage,", '')
        data_string = data_string.replace("code,", '')

        try:
            channel, timestamp, voltage, code = data_string.split(",")

            # Add data to lists
            if channel == "0":
                ch0_voltages.append(round(float(voltage), 4))
                ch0_values.append(self.convert_accelerometer(float(voltage)))

                # Limit lists to the max elements value
                ch0_timestamps = ch0_timestamps[-max_elements:]
                ch0_voltages = ch0_voltages[-max_elements:]
                ch0_values = ch0_values[-max_elements:]

            elif channel == "4":
                ch4_voltages.append(round(float(voltage) * 1000, 4))
                ch4_values.append(self.convert_temperature(float(code)))

                # Limit lists to the max elements value
                ch4_timestamps = ch4_timestamps[-max_elements:]
                ch4_voltages = ch4_voltages[-max_elements:]
                ch4_values = ch4_values[-max_elements:]

            elif channel == "6":
                ch6_voltages.append(round(float(voltage), 4))
                ch6_values.append(self.convert_pressure(float(voltage)))

                # Limit lists to the max elements value
                ch6_timestamps = ch6_timestamps[-max_elements:]
                ch6_voltages = ch6_voltages[-max_elements:]
                ch6_values = ch6_values[-max_elements:]

        except Exception as e:
            logging.exception(e)
            pass

            
    def on_serial_port_change(self, change):
        if change['type'] == 'change' and change['name'] == 'value':
            self.selected_serial_port = change['new']            

    def __serial_port_open(self):
        """ Check if the serial port is open.

        :returns:
            True if the serial port is open.
            False if the serial port is closed.
        """

        if self.serial:
            if self.serial.isOpen():
                return True

        return False

    def __serial_port_scan(self):
        """ Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
        """
        if sys.platform.startswith('win'):
            ports = ['COM%s' % (i + 1) for i in range(256)]
        elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
            # this excludes your current terminal "/dev/tty"
            ports = glob.glob('/dev/tty[A-Za-z]*')
        elif sys.platform.startswith('darwin'):
            ports = glob.glob('/dev/tty.*')
        else:
            raise EnvironmentError('Unsupported platform')

        result = []
        for port in ports:
            try:
                self.serial = pyserial.Serial(port)
                self.serial.close()
                result.append(port)
            except (OSError, pyserial.SerialException):
                pass

        return result
    
    def __serial_ctrl_c(self):
        """ Send a ctrl+c command to the terminal

        :return: Nothing
        """
        if self.__serial_port_open():
            self.serial.write("\x03\n".encode('utf-8'))
            
    def __login(self):
        """ Login to the device

        :return: Nothing
        """
        if self.__serial_port_open():
            print("Logging in as root...")
            self.serial.write("root\n".encode('utf-8'))
            
    def safari_start(self):
        """ Start the Safari script on the device.

        :return: Nothing
        """
        global safari_running
        
        safari_running = True
        self.serial.write("python /root/python/safari.py\n".encode('utf-8'))

    def safari_stop(self):
        """ Stop the Safari script, or any running script, on the device

        :return: Nothing
        """
        global safari_running
        
        safari_running = False
        test_mode_enabled = False
        self.__serial_ctrl_c()
        
    @staticmethod
    def convert_pressure(voltage):
        """ Convert the supplied voltage to Pascals

        :param voltage: Voltage to convert
        :return: Pressure measured in Pascals (Pa)


        DP = (190 * Vmeas)/Vdd - 38Pa
        Example:
        Vmeas = 875mV, Vdd = 3.3V
        Differential Pressure = (190 * 0.875V)/3.3V - 38Pa = 12.37Pa
        """
        vcc = 3.3
        r1 = 132000
        r2 = 100000
        return round(((190.0 * voltage * (r1/r2)) / vcc - 38), 3)

    @staticmethod
    def convert_accelerometer(voltage):
        """ Convert the supplied voltage to g's

        :param voltage: Voltage to convert
        :return: Acceleration measured in g's


        Acceleration (g's) = (Vmeas-VCC/2) * 1g/620mV
        Example:
        Vmeas = 2.05V, Vcc = 3.3V
        Acceleration = (2.05V-1.65V) * 1g/0.620V = 0.645g
        """
        vcc = 3.3
        return round((voltage - vcc / 2) * 1 / 0.640, 3)

    @staticmethod
    def convert_temperature(code):
        """ Convert the supplied code to temperature

        :param code: Raw ADC value received from AD7124
        :return: Temperature in degrees Celsius.


        RTD Resistance = (Code * 5.11kOhms) / ((2^24) * 16)
        Temperature (Degrees C) = (RTD Resistance - 100ohms) / (0.385 ohms/degC)
        """
        r_rtd = (code * 5110) / ((2**24) * 16)
        temp = (r_rtd - 100) / 0.385
        return round(temp, 2)

In [ ]:
class Data:   
    def __init__(self):
        
        self.update_interval = 10  # Time (ms) between polling/animation updates   
        self.fig, (self.ax1, self.ax2, self.ax3)  = plt.subplots(3, 1)
        
        plt.get_current_fig_manager().window.state('zoomed')
        
        color = 'tab:blue'
        self.ax1.set_title("Channel 2", fontsize=12)
        self.ax1.set_ylabel('Voltage (V)', color=color)
        self.ax1.tick_params(axis='y', labelcolor=color)

        color = 'tab:red'
        self.ax2.set_title("Channel 4")
        self.ax2.set_ylabel('Voltage (V)', color=color)
        self.ax2.tick_params(axis='y', labelcolor=color)

        color = 'tab:green'
        self.ax3.set_title("Channel 6")
        self.ax3.set_ylabel('Voltage (V)', color=color)
        self.ax3.tick_params(axis='y', labelcolor=color)
        
        self.annot_ax1 = self.ax1.annotate("", xy=(0, 0), xytext=(-20, 20), textcoords="offset points",
                                           bbox=dict(boxstyle="round", fc="w"),
                                           arrowprops=dict(arrowstyle="->"))
        self.annot_ax2 = self.ax2.annotate("", xy=(0, 0), xytext=(-20, 20), textcoords="offset points",
                                           bbox=dict(boxstyle="round", fc="w"),
                                           arrowprops=dict(arrowstyle="->"))
        self.annot_ax3 = self.ax3.annotate("", xy=(0, 0), xytext=(-20, 20), textcoords="offset points",
                                           bbox=dict(boxstyle="round", fc="w"),
                                           arrowprops=dict(arrowstyle="->"))

        self.annot_ax1_visible = False
        self.annot_ax2_visible = False
        self.annot_ax3_visible = False

        self.annot_ax1_text = ""
        self.annot_ax2_text = ""
        self.annot_ax3_text = ""

        self.line_ax1 = None
        self.line_ax2 = None
        self.line_ax3 = None
        self.annot_ax1_xy = None
        self.annot_ax2_xy = None
        self.annot_ax3_xy = None

        # https://bit.ly/2AWOqiu
        self.fig.canvas.mpl_connect("motion_notify_event", self.hover)
        
        self.ani = animation.FuncAnimation(self.fig,
                                           self.animate,
                                           interval=self.update_interval)
        HTML(self.ani.to_jshtml())
        
        self.thread = threading.Thread(target=self.animate_data)
        self.thread.daemon = True
            
        #self.thread.start()

    # https://bit.ly/2AWOqiu
    # Function for updating the hovering annotations
    def update_annot(self, axis, ind):
        if axis == self.ax1:
            x, y = self.line_ax1.get_data()
            self.annot_ax1_xy = (x[ind["ind"][0]], y[ind["ind"][0]])
            self.annot_ax1_text = "{}".format(y[ind["ind"][0]])
        elif axis == self.ax2:
            x, y = self.line_ax2.get_data()
            self.annot_ax2_xy = (x[ind["ind"][0]], y[ind["ind"][0]])
            self.annot_ax2_text = "{}".format(y[ind["ind"][0]])
        elif axis == self.ax3:
            x, y = self.line_ax3.get_data()
            self.annot_ax3_xy = (x[ind["ind"][0]], y[ind["ind"][0]])
            self.annot_ax3_text = "{}".format(y[ind["ind"][0]])

    # Callback for the hover event
    def hover(self, event):
        vis_ax1 = self.annot_ax1.get_visible()
        vis_ax2 = self.annot_ax2.get_visible()
        vis_ax3 = self.annot_ax3.get_visible()
        try:
            if event.inaxes == self.ax1:
                if self.line_ax1:
                    cont, ind = self.line_ax1.contains(event)
                    if cont:
                        self.update_annot(self.ax1, ind)
                        self.annot_ax1_visible = True
                    else:
                        if vis_ax1:
                            self.annot_ax1_visible = False

            elif event.inaxes == self.ax2:
                if self.line_ax2:
                    cont, ind = self.line_ax2.contains(event)
                    if cont:
                        self.update_annot(self.ax2, ind)
                        self.annot_ax2_visible = True
                    else:
                        if vis_ax2:
                            self.annot_ax2_visible = False

            if event.inaxes == self.ax3:
                if self.line_ax3:
                    cont, ind = self.line_ax3.contains(event)
                    if cont:
                        self.update_annot(self.ax3, ind)
                        self.annot_ax3_visible = True
                    else:
                        if vis_ax3:
                            self.annot_ax3_visible = False
        except IndexError:
            print("Uh-Oh")
        
    def animate(self, i):
        global ch0_timestamps
        global ch4_timestamps
        global ch6_timestamps

        global ch0_voltages
        global ch4_voltages
        global ch6_voltages

        global ch0_values
        global ch4_values
        global ch6_values
        
        ax1_color = 'tab:blue'
        ax2_color = 'tab:red'
        ax3_color = 'tab:green'
        plot_title_fontsize = 12
        plot_label_fontsize = 12

        self.ax1.clear()
        self.ax2.clear()
        self.ax3.clear()
        self.ax1.tick_params(axis='y', labelcolor=ax1_color)
        self.ax2.tick_params(axis='y', labelcolor=ax2_color)
        self.ax3.tick_params(axis='y', labelcolor=ax3_color)

        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

        # Clear, format, and plot voltage values
        self.ax1.set_title("Channel 0 Voltage", fontsize=plot_title_fontsize)
        self.ax1.set_ylabel('Voltage (V)', color=ax1_color, fontsize=plot_label_fontsize)
        self.ax2.set_title("Channel 4 Voltage", fontsize=plot_title_fontsize)
        self.ax2.set_ylabel('Voltage (mV)', color=ax2_color, fontsize=plot_label_fontsize)
        self.ax3.set_title("Channel 6 Voltage", fontsize=plot_title_fontsize)
        self.ax3.set_ylabel('Voltage (V)', color=ax3_color, fontsize=plot_label_fontsize)
        self.ax1.yaxis.set_major_formatter(FormatStrFormatter('%.4f'))
        self.ax2.yaxis.set_major_formatter(FormatStrFormatter('%.4f'))
        self.ax3.yaxis.set_major_formatter(FormatStrFormatter('%.4f'))

        try:
            if len(ch0_voltages) > 0:
                self.ax1.plot(np.arange(len(ch0_voltages)), ch0_voltages,
                              linewidth=2, color=ax1_color)
                self.line_ax1 = self.ax1.get_lines()[0]
                self.annot_ax1 = self.ax1.annotate("", xy=(0, 0), xytext=(-20, 20), textcoords="offset points",
                                                   bbox=dict(boxstyle="round", fc="w"),
                                                   arrowprops=dict(arrowstyle="->"))
                self.annot_ax1.set_visible(self.annot_ax1_visible)
                self.annot_ax1.xy = self.annot_ax1_xy
                self.annot_ax1.set_text(self.annot_ax1_text)
                self.annot_ax1.get_bbox_patch().set_alpha(0.4)
                try:
                    ax1_min = min(ch0_voltages)
                    ax1_mean = sum(ch0_voltages) / len(ch0_voltages)
                    ax1_max = max(ch0_voltages)
                    ax1_last = ch0_voltages[-1]
                    ax1_annotation = '\n'.join((
                        r'Max=%.4f' % (ax1_max,),
                        r'Mean=%.4f' % (ax1_mean,),
                        r'Min=%.4f' % (ax1_min,),
                        r'Last=%.4f' % (ax1_last,)))

                    self.ax1.text(1.01, 0.7, ax1_annotation, transform=self.ax1.transAxes, fontsize=10,
                                  verticalalignment='top', bbox=props)
                except ValueError:
                    print("Dimension Error")

            if len(ch4_voltages) > 0:
                self.ax2.plot(np.arange(len(ch4_voltages)), ch4_voltages,
                                  linewidth=2, color=ax2_color)
                self.line_ax2 = self.ax2.get_lines()[0]
                self.annot_ax2 = self.ax2.annotate("", xy=(0, 0), xytext=(-20, 20), textcoords="offset points",
                                                   bbox=dict(boxstyle="round", fc="w"),
                                                   arrowprops=dict(arrowstyle="->"))
                self.annot_ax2.set_visible(self.annot_ax2_visible)
                self.annot_ax2.xy = self.annot_ax2_xy
                self.annot_ax2.set_text(self.annot_ax2_text)
                self.annot_ax2.get_bbox_patch().set_alpha(0.4)
                try:
                    ax2_min = min(ch4_voltages)
                    ax2_mean = sum(ch4_voltages) / len(ch4_voltages)
                    ax2_max = max(ch4_voltages)
                    ax2_last = ch4_voltages[-1]
                    ax2_annotation = '\n'.join((
                        r'Max=%.4f' % (ax2_max,),
                        r'Mean=%.4f' % (ax2_mean,),
                        r'Min=%.4f' % (ax2_min,),
                        r'Last=%.4f' % (ax2_last,)))
                    self.ax2.text(1.01, 0.7, ax2_annotation, transform=self.ax2.transAxes, fontsize=10,
                                  verticalalignment='top', bbox=props)
                except ValueError:
                    print("Dimension Error")

            if len(ch6_voltages) > 0:
                self.ax3.plot(np.arange(len(ch6_voltages)), ch6_voltages,
                                  linewidth=2, color=ax3_color)
                self.line_ax3 = self.ax3.get_lines()[0]
                self.annot_ax3 = self.ax3.annotate("", xy=(0, 0), xytext=(-20, 20), textcoords="offset points",
                                                   bbox=dict(boxstyle="round", fc="w"),
                                                   arrowprops=dict(arrowstyle="->"))
                self.annot_ax3.set_visible(self.annot_ax3_visible)
                self.annot_ax3.xy = self.annot_ax3_xy
                self.annot_ax3.set_text(self.annot_ax3_text)
                self.annot_ax3.get_bbox_patch().set_alpha(0.4)
                try:
                    ax3_min = min(ch6_voltages)
                    ax3_mean = sum(ch6_voltages) / len(ch6_voltages)
                    ax3_max = max(ch6_voltages)
                    ax3_last = ch6_voltages[-1]
                    ax3_annotation = '\n'.join((
                        r'Max=%.4f' % (ax3_max,),
                        r'Mean=%.4f' % (ax3_mean,),
                        r'Min=%.4f' % (ax3_min,),
                        r'Last=%.4f' % (ax3_last,)))
                    self.ax3.text(1.01, 0.7, ax3_annotation, transform=self.ax3.transAxes, fontsize=10,
                                  verticalalignment='top', bbox=props)
                except ValueError:
                    print("Dimension Error")

        except Exception as e:
            logging.exception(e)
            print("Dimension Error")

        if len(ch4_timestamps) > 0:
            try:
                ax2.xaxis.set_major_formatter(mdates.DateFormatter('%M:%S.%f'))
            except Exception as e:
                logging.exception(e)
                pass

        if len(ch6_timestamps) > 0:
            try:
                ax3.xaxis.set_major_formatter(mdates.DateFormatter('%M:%S.%f'))
            except Exception as e:
                logging.exception(e)
                pass

In [ ]:
#if __name__ == '__main__':

#Initialize the classes
serial = Serial()
display(widgets.HBox([serial.serial_port_list, serial.serialButton]))

In [ ]:
data = Data()

In [ ]:
serial.safari_start()